Adicionando o banco

In [21]:
import pandas as pd

df = pd.read_csv("futevolei.csv", encoding="latin-1")


print(df.head())


   Partida                Jogadore(a)s Resultado  Unnamed: 3  Unnamed: 4  \
0        1    romario, roberto, Alonso    Perdeu         NaN         NaN   
1        2      Alonso, roberto, ?gata    Perdeu         NaN         NaN   
2        3       ?gata, Ronaldo, Marta    GANHOU         NaN         NaN   
3        4       ?gata, Marta, B rbara    GANHOU         NaN         NaN   
4        5  B rbara, Rob'erto, Romario    Perdeu         NaN         NaN   

  Jogadore(a)s.1  
0        Romario  
1        Roberto  
2          Ágata  
3          Marta  
4        Bárbara  


In [ ]:
import pandas as pd
import unicodedata, re

def strip_accents(txt: str) -> str:
    txt = str(txt)
    nfkd = unicodedata.normalize("NFKD", txt)
    return "".join(ch for ch in nfkd if not unicodedata.combining(ch))

def base_key(txt: str) -> str:
    # chave minúscula, sem acentos e só letras (para comparar)
    return re.sub(r"[^a-z]", "", strip_accents(txt).lower())

# nomes
canon = {
    "romario": "Romário",
    "roberto": "Roberto",
    "ronaldo": "Ronaldo",
    "alonso":  "Alonso",
    "agata":   "Ágata",
    "gata":    "Ágata",    
    "marta":   "Marta",
    "barbara": "Bárbara",
    "brbara":  "Bárbara",  
}

def normaliza_nome(n):
    n = str(n).strip().replace("?", " ")
    k = base_key(n)
    # se reconhecer no dicionário, usa o nome canônico; senão, aplica Title Case
    return canon.get(k, n.title())

def normaliza_campo_jogadores(campo):
    s = str(campo)
    # separadores mais comuns: vírgula ou ponto e vírgula
    partes = re.split(r"[;,]", s)
    partes = [p.strip() for p in partes if p.strip()]
    if len(partes) <= 1:
        return normaliza_nome(s)
    return ", ".join(normaliza_nome(p) for p in partes)

# limpar cabeçalhos e achar as colunas de jogadores
df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]  # remove colunas vazias
df.columns = [c.strip().replace("\ufeff", "") for c in df.columns]

# pega todas as colunas cujo nome (sem acento) contenha "jogador"
cols_jogadores = [c for c in df.columns if "jogador" in strip_accents(c).lower()]

# aplica a normalização em todas as colunas de jogadores
for c in cols_jogadores:
    df[c] = df[c].apply(normaliza_campo_jogadores)

# padroniza a coluna Resultado
if "Resultado" in df.columns:
    df["Resultado"] = (df["Resultado"].astype(str)
                       .str.upper().str.strip()
                       .replace({"GANHOU":"GANHOU", "PERDEU":"PERDEU"}))

# salvar o CSV tratado
df.to_csv("futevolei_utf8.csv", index=False, encoding="utf-8-sig")

df.head()


,Partida,Jogadore(a)s,Resultado,Jogadore(a)s.1
0,1,"Romário, Roberto, Alonso",PERDEU,Romário
1,2,"Alonso, Roberto, Ágata",PERDEU,Roberto
2,3,"Ágata, Ronaldo, Marta",GANHOU,Ágata
3,4,"Ágata, Marta, Bárbara",GANHOU,Marta
4,5,"Bárbara, Roberto, Romário",PERDEU,Bárbara
